
# MTH3302 : Méthodes probabilistes et statistiques pour l'I.A.

Jonathan Jalbert<br/>
Professeur agrégé au Département de mathématiques et de génie industriel<br/>
Polytechnique Montréal<br/>

# Projet A2021 : Prédire les maladies cardiaques

La description du projet est disponible à l'adresse suivante :
https://www.kaggle.com/t/3a185493d8bb48d8961ad50e465bed4f

Ce calepin Jupyter de base permet de 

1. Charger les données fournies.
2. Effectuer une analyse exploratoire sommaire des données.
3. Développer un modèle simple de prédiction.
4. Générer le fichier de prédictions à téléverser sur Kaggle.

Dans un premier temps, vous devrez récupérer les données sous l'onglet *data* du site Kaggle. Il y a deux fichiers :
- train.csv
- test.csv

Le fichier *train.csv* contient les données sur lesquelles vous pouvez entraîner votre modèle. Il sera ensuite évaluée sur les données de l'ensemble *test.csv* lorsque vous aurez téléversé vos prédictions sur Kaggle. 

### Consignes

- Vous devez constituer une équipe de 3 à 5 personnes.
- Au moins une solution doit être proposée sur Kaggle.
- Utilisez votre numéro d'équipe pour téléverser vos prédictions sur Kaggle.
- Un seul fichier .ipynb par équipe faisant office de rapport et permettant de reproduire vos meilleures prédictions doit être remis.
- Le langage Julia doit être utilisé.
- Votre démarche doit être rigoureusement justifiée (consultez la grille de correction pour vous orienter).

### Quelques conseils

Votre calepin doit permettre de suivre clairement votre raisonnement et de reproduire vos résultats. Garder à l'esprit que vos résultats et votre démarche doivent être reproductibles par une personne à l'extérieur de votre équipe. Le calepin constitue le rapport. Servez vous des cellules de texte pour décrire ce que vous faites.

Je vous encourage fortement à faire une analyse exploratoire de vous données pour développer une meilleure expertise du problème. C'est une étape qui est toujours négligée par les débutants mais qui est essentielle. C'est avec l'analyse exploratoire que vous viendra des idées d'amélioration, par exemple créer de nouvelles variables explicatives.

Vous pouvez utiliser directement tout ce qui se retrouve dans les notes de cours sans explication et toutes les librairies utilisées dans le cours (incluant mes fonctions).

Ce calepin de base contient un modèle très simple de prédiction : on prédit 0 débordement à tous les jours. Ce sera votre travail d'améliorer ces prédictions naïves avec la méthode de votre choix.

Il faudra que vous trouviez un moyen de traiter les données manquantes. La plupart du temps, une méthode simple d'imputation (de remplacement) des données manquantes est appropriée.

Prenez la peine de tout documenter, même les essais infructueux. Ce n'est pas nécessaire de les expliquer en détails, mais c'est important de les mentionner au moins succintement dans la discussion avec une raison possible pour leur échec. De cette façon, une personne qui reprendra votre travail dans le futur ne perdra pas de temps à réessayer une méthode déjà implémentée et infructueuse.

Vous pouvez aussi indiquer dans votre rapport les raisons qui vous font croire pourquoi une méthode à moins bien performée que ce à quoi vous vous attendiez. Vous pouvez également mentionner ce que vous auriez pu tenter si vous aviez eu plus de temps ou plus de données, etc. L'idée est de guider le prochain scientifique qui prendra la relève de votre travail.

Vous êtes limités à deux soumissions par jour par équipe sur Kaggle. Je vous suggère donc de bien tester vos modèles localement et de ne téléverser que vos meilleurs candidats.

In [ ]:
using CSV, DataFrames, Gadfly, GLM, Statistics, LinearAlgebra, Distributions, Combinatorics, StatsBase, MLBase

In [ ]:
include("functions.jl");

## 1. Chargement des données

Assurez vous d'avoir télécharger les données dans le répertoire de ce calepin.

In [ ]:
data = CSV.read("train1.csv", DataFrame)
first(data,5)

select!(data, Not([:ExerciseAngina, :Oldpeak, :Cholesterol]))
y = data[:, :HeartDisease]
n = length(y)

In [ ]:
cholesterol_mean = floor(Int, mean(skipmissing(data.Cholesterol)))
data[!, :Cholesterol] = coalesce.(data.Cholesterol, cholesterol_mean);

In [ ]:
data.Cholesterol = replace(data.Cholesterol, missing => cholesterol_mean);
# dropmissing!(data)

In [ ]:
sex = zeros(Int, n)

for i in 1:n
    if (data.Sex[i] == "M")
        sex[i] = 1;
    end
end
data[!, :Sex] = sex;
data

In [ ]:
ChestPainType1 = zeros(Int, n)
ChestPainType2 = zeros(Int, n)
ChestPainType3 = zeros(Int, n)

for i in 1:n
    if (data.ChestPainType[i] == "ATA")
        ChestPainType1[i] = 1;
    elseif (data.ChestPainType[i] == "NAP")
        ChestPainType2[i] = 1; 
    elseif (data.ChestPainType[i] == "ASY")
        ChestPainType3[i] = 1;
    end
end
select!(data, Not(:ChestPainType))
data[!, :ChestPainType1] = ChestPainType1;
data[!, :ChestPainType2] = ChestPainType2;
data[!, :ChestPainType3] = ChestPainType3;

In [ ]:
restingECG1 = zeros(Int, n)
restingECG2 = zeros(Int, n)

for i in 1:n
    if (data.RestingECG[i] == "ST")
        restingECG1[i] = 1;
    elseif (data.RestingECG[i] == "LVH")
        restingECG2[i] = 1;  
    end
end
select!(data, Not(:RestingECG))
data[!, :RestingECG1] = restingECG1;
data[!, :RestingECG2] = restingECG2;

In [ ]:
exerciseAngina = zeros(Int, n)

for i in 1:n
    if (data.ExerciseAngina[i] == "Y")
        exerciseAngina[i] = 1;
    end
end
data[!, :ExerciseAngina] = exerciseAngina;

In [ ]:
STSlope1 = zeros(Int, n)
STSlope2 = zeros(Int, n)

for i in 1:n
    if (data.STSlope[i] == "Flat")
        STSlope1[i] = 1;
    elseif (data.STSlope[i] == "Down")
        STSlope2[i] = 1;  
    end
end
select!(data, Not(:STSlope))
data[!, :STSlope1] = STSlope1;
data[!, :STSlope2] = STSlope2;



In [ ]:
countmap(data.HeartDisease)

In [ ]:
data_temp = select(data, Not([:ID, :HeartDisease]))

dt = StatsBase.fit(StatsBase.ZScoreTransform, Matrix{Float64}(data_temp), dims=1)
Z = StatsBase.transform(dt, Matrix{Float64}(data_temp))

dt = StatsBase.fit(StatsBase.ZScoreTransform, Vector{Float64}(data.HeartDisease), dims=1)
Y = StatsBase.transform(dt, Vector{Float64}(data.HeartDisease))

F = svd(Z)
U = F.U
V = F.V
γ = F.S

T = Z*V

df = DataFrame(T, ["Col" * string(i) for i = 1:size(T, 2)])
df.ID = data.ID
df.HeartDisease = data.HeartDisease

# df

In [ ]:
variables = propertynames(select(df, Not([:ID, :HeartDisease])))

df_auc = DataFrame(Variable = Vector{Symbol}[], AUC = Float64[])

for variable in combinations(variables)
    formula = term(:HeartDisease) ~ sum(term.(variable))
    
    M_temp = glm(formula, df,  Bernoulli(), LogitLink())
    θ̂ = convert(Array{Float64}, predict(M_temp, df))
    aireRoc = auc(df.HeartDisease, θ̂ )
    
    push!(df_auc, [variable, aireRoc])
end

models = sort(df_auc, :AUC, rev=true)


In [ ]:
formula = term(:HeartDisease) ~ sum(term.(models.Variable[1])[i] for i=1:length(models.Variable[1]))
M = glm(formula, df, Bernoulli(), LogitLink())

In [ ]:
M = glm(@formula(HeartDisease ~ Col1 + Col2 + Col3 + Col4 + Col5 + Col7 + Col9), df, Bernoulli(), LogitLink())

## 2. Analyse exploratoire sommaire

C'est une analyse exploratoire sommaire. Je vous encourage fortement à poursuivre cette analyse.

#### 2.1 Maladie cardiovasculaire en fonction du rythme cardiaque maximal

In [ ]:
# Calcul de la moyenne par classe

combine(groupby(data, :HeartDisease), :MaxHR => mean)

In [ ]:
# Affichage des rythmes cardiques maximum en fonction de la classe

df = select(data, :MaxHR, :HeartDisease)

df.HeartDisease = string.(df.HeartDisease)

plot(df, x=:HeartDisease, y=:MaxHR, Geom.boxplot)

In [ ]:
# Affichage des rythmes cardiques maximum en fonction de l'age

df = select(data, :Age, :HeartDisease)

df.HeartDisease = string.(df.HeartDisease)

plot(df, x=:HeartDisease, y=:Age, Geom.boxplot)

## 3. Ajustement d'un modèle de régression logistique

Ici, je n'utilise que la rythme cardiaque maximum comme variable explicative.

In [ ]:
# variables = propertynames(select(data, Not([:ID, :HeartDisease, :Cholesterol, :RestingBP, :RestingECG, :MaxHR])))
# formula = term(:HeartDisease) ~ sum(Term(variables[i]) for i = 1:length(variables))
formula = term(:HeartDisease) ~ sum(term.(models.Variable[6])[i] for i=1:length(models.Variable[1]))
# formula = @formula(HeartDisease ~ Age + Sex + FastingBS + MaxHR + ExerciseAngina + Oldpeak + ChestPainType3 + STSlope1 + STSlope2)
# formula = @formula(formula)
M = glm(formula, data, Bernoulli(), LogitLink())


## 4. Prédiction des surverses pour les jours de l'ensemble de test

On utilise le modèle simple de la section précédente pour estimer la probabilité que le patient souffre d'une maladie cardiovasculaire.

#### 4.1 Chargement des données de l'ensemble de test

In [ ]:
test = CSV.read("test1.csv", DataFrame);
y = test.HeartDisease
n = length(y)

In [ ]:
select!(test, Not([:ExerciseAngina, :Oldpeak, :STSlope, :Cholesterol]))


In [ ]:
cholesterol_mean = floor(Int, mean(skipmissing(test.Cholesterol)))
test[!, :Cholesterol] = coalesce.(test.Cholesterol, cholesterol_mean);
test.Cholesterol = replace(test.Cholesterol, missing => cholesterol_mean);
# dropmissing!(data)

In [ ]:
sex = zeros(Int, n)

for i in 1:n
    if (test.Sex[i] == "M")
        sex[i] = 1;
    end
end
test[!, :Sex] = sex;

In [ ]:
ChestPainType1 = zeros(Int, n)
ChestPainType2 = zeros(Int, n)
ChestPainType3 = zeros(Int, n)

for i in 1:n
    if (test.ChestPainType[i] == "ATA")
        ChestPainType1[i] = 1;
    elseif (test.ChestPainType[i] == "NAP")
        ChestPainType2[i] = 1; 
    elseif (test.ChestPainType[i] == "ASY")
        ChestPainType3[i] = 1;
    end
end
select!(test, Not(:ChestPainType))
test[!, :ChestPainType1] = ChestPainType1;
test[!, :ChestPainType2] = ChestPainType2;
test[!, :ChestPainType3] = ChestPainType3;

In [ ]:
restingECG1 = zeros(Int, n)
restingECG2 = zeros(Int, n)

for i in 1:n
    if (test.RestingECG[i] == "ST")
        restingECG1[i] = 1;
    elseif (test.RestingECG[i] == "LVH")
        restingECG2[i] = 1;  
    end
end
select!(test, Not(:RestingECG))
test[!, :RestingECG1] = restingECG1;
test[!, :RestingECG2] = restingECG2;

In [ ]:
exerciseAngina = zeros(Int, n)

for i in 1:n
    if (test.ExerciseAngina[i] == "Y")
        exerciseAngina[i] = 1;
    end
end
test[!, :ExerciseAngina] = exerciseAngina;

In [ ]:
STSlope1 = zeros(Int, n)
STSlope2 = zeros(Int, n)

for i in 1:n
    if (test.STSlope[i] == "Flat")
        STSlope1[i] = 1;
    elseif (test.STSlope[i] == "Down")
        STSlope2[i] = 1;  
    end
end
select!(test, Not(:STSlope))
test[!, :STSlope1] = STSlope1;
test[!, :STSlope2] = STSlope2;

In [ ]:
test_temp = select(test, Not([:ID, :HeartDisease]))
dt = StatsBase.fit(StatsBase.ZScoreTransform, Matrix{Float64}(test_temp), dims=1)
Z_test = StatsBase.transform(dt, Matrix{Float64}(test_temp))

nameVarExpPrincipal = [:Col1, :Col2, :Col3,:Col4, :Col5, :Col6, :Col7, :Col8, :Col9,:Col10]
T_test = Z_test*V 
dfPrincipalTest = DataFrame(T_test, nameVarExpPrincipal)

M = glm(@formula(HeartDisease ~ Col1 + Col2 + Col3 + Col4 + Col5 + Col7 + Col9), df, Bernoulli(), LogitLink())

#### 4.2 Prédiction pour chacun des patients de l'ensemble de test

On prédit que le patient souffre d'une maladie cardiovasculaire si la probabilité est supérieure à 50%.

In [ ]:
θ̂ = predict(M, dfPrincipalTest)

ŷ = Int64[]
index = 0;
for θ̂ᵢ in θ̂
    if θ̂ᵢ >= .5
        push!(ŷ, 1)
    else
        push!(ŷ, 0)
    end
end


#### 3.3 Préparation du fichier des préditions pour téléverser sur Kaggle

Le fichier *benchmark_predictions.csv* généré peut être téléversé sur Kaggle. Il est composé d'une colonne d'identifiants (ID) et d'une colonne des diagnostics prédits.

In [ ]:
prediction = DataFrame(ID = test.ID, Prediction = ŷ)

CSV.write("benchmark_predictions.csv", prediction)

In [ ]:
count = 0;

for i = 1:nrow(prediction)
   if (prediction[i, 2] == test.HeartDisease[i])
        count = count + 1;
    end 
end
    
count / nrow(prediction)
